In [2]:
from requests_html import HTMLSession # for making request
import requests # for making request
import json # for API parsing
import pandas as pd # for data processing
import numpy as np # for data processing
from tqdm import tqdm # for count time of iteration
import re
from tqdm import tqdm
from imdb import IMDb
from imdb.Person import Person

# Load set id

In [3]:
movie_df = pd.read_csv('./data/movie.csv', dtype={'id':str})

In [41]:
movie_df.to_csv('./data/movie.csv', index=False)

In [4]:
set_id = set(movie_df['id'])

In [8]:
set_id_small = ['3495026',
 '1504320',
 '0099939',
 '0085334',
 '0039689',
 '0190641',
 '5886046',
 '0108162',
 '0071249',
 '0110737',
 '0101698',
 '0070130',
 '0108122',]

# Parse HTML

In [9]:
session = HTMLSession()

In [10]:
users = {}
for id in tqdm(set_id_small):
    try:
        user_rating = {}
        for i in range(1,11,2):
            url = f'https://www.imdb.com/title/tt{id}/reviews?sort=reviewVolume&dir=desc&ratingFilter={i}'
            r = session.get(url)
            try:
                user = r.html.find('.display-name-link > a')
                user_rating[i] = user
            except:
                pass
        users[id] = user_rating
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [01:42<00:00,  7.90s/it]


# Save data

In [27]:
def get_id_from_element(ele):
    id = re.search('/ur(.+?)/', ele.links.pop())
    if id is None:
        return None
    return id.group(1)

In [33]:
ratings = []
for key_id in tqdm(users):
    for key_rating in users[key_id]:
        for element in users[key_id][key_rating]:
            dict_row = {}
            user_id = get_id_from_element(element)
            dict_row['movie_id'] = key_id
            dict_row['user_id'] = user_id
            dict_row['rating'] = int(key_rating)
            ratings.append(dict_row)

100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 343.01it/s]


In [37]:
ratings_df = pd.DataFrame(data=ratings)

In [42]:
ratings_df.to_csv('./data/rating.csv', index=False)

In [ ]:
ratings_df = []
for i in range(1,11):
    df_